In [232]:
import numpy as np
import pandas as pd
import math 

In [233]:
chess_data = pd.read_csv('/w/225/1/chess/tactics/glicko_user_tactics_problem.csv_00')

In [234]:
#chess_data.head()

In [235]:
passed_only = chess_data[chess_data["is_passed"] == 1]

In [236]:
#passed_only

In [237]:
vocab_dat = passed_only.groupby(["tactics_problem_id"], sort = False).size()

In [238]:
#vocab_dat.head()

In [239]:
count = np.int64(vocab_dat)
problem_id = pd.unique(passed_only['tactics_problem_id'])
dict_id_count = dict(zip(problem_id,count))

In [240]:
sampling_size = 1000000
PPMI_table = pd.DataFrame(index=np.arange(sampling_size), columns=['puzz1','puzz2','p1','p2','p1p2','PPMI'])

In [241]:
#PPMI_table.head()

In [243]:
sample_pairs = np.random.choice(problem_id, size = (sampling_size,2), replace = True)
PPMI_table.iloc[:,0:2] = pd.DataFrame(sample_pairs, columns=['puzz1','puzz2'])

In [244]:
#PPMI_table.head()

In [245]:
puzz1 = np.array(PPMI_table['puzz1'])
puzz2 = np.array(PPMI_table['puzz2'])
PPMI_table['p1'] = np.vectorize(dict_id_count.get)(puzz1)
PPMI_table['p2'] = np.vectorize(dict_id_count.get)(puzz2)

In [246]:
#PPMI_table.head()

In [247]:
temp = passed_only[['tactics_problem_id','user_hash']]

In [248]:
#temp

In [249]:
def f(df):
    keys, values = df.sort_values('tactics_problem_id').values.T
    ukeys, index = np.unique(keys, True)
    arrays = np.split(values, index[1:])
    user_hash = [list(a) for a in arrays]
    return dict(zip(ukeys,user_hash))

temp2 = f(temp)

In [250]:
p1_hash = np.vectorize(temp2.get, otypes = [list])(puzz1)
p2_hash = np.vectorize(temp2.get, otypes = [list])(puzz2)

In [ ]:
for i in range(p1_hash.size):
    p1 = np.array(p1_hash[i])
    p2 = np.array(p2_hash[i])
    p1p2 = np.intersect1d(p1,p2)
    PPMI_table.iloc[i,4] = p1p2.size
    if(PPMI_table.iloc[i,4] != 0):
        PPMI_table.iloc[i,5] = max(np.log(np.array(PPMI_table.iloc[i,4])) + math.log(vocab_dat.size) - np.log(np.array(PPMI_table.iloc[i,2])) - np.log(np.array(PPMI_table.iloc[i,3])),0)
    else:
        PPMI_table.iloc[i,5] = 0

In [ ]:
#PPMI_table.head()